In [1]:
import sqlite3, pandas as pd
conn = sqlite3.connect("db/seoul_bus_172.db")

print(pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn))

print(pd.read_sql("""
  SELECT date, route_no, stop_id, stop_name, board_total, alight_total
  FROM fact_demand_daily_stop
  ORDER BY date, stop_id
  LIMIT 5;
""", conn))

print(pd.read_sql("""
  SELECT use_ym, route_no, stop_id, hour, board_total, alight_total
  FROM fact_demand_monthly_hourly_stop
  ORDER BY stop_id, hour
  LIMIT 5;
""", conn))

conn.close()

                              name
0                    dim_stop_link
1           fact_demand_daily_stop
2  fact_demand_monthly_hourly_stop
3                    dim_route_map
       date route_no    stop_id             stop_name  board_total  \
0  20250601      172  100000002    창경궁.서울대학교병원(00028)           42   
1  20250601      172  100000003      명륜3가.성대입구(00027)          129   
2  20250601      172  100000005  혜화동로터리.여운형활동터(00026)           69   
3  20250601      172  100000076    창덕궁.우리소리박물관(00029)          105   
4  20250601      172  100000103    창덕궁.우리소리박물관(00078)          162   

   alight_total  
0            67  
1           115  
2           184  
3           187  
4           147  
   use_ym route_no    stop_id  hour  board_total  alight_total
0  202506      172  100000002     0            0             0
1  202506      172  100000002     1            0             0
2  202506      172  100000002     2            0             0
3  202506      172  100000002     3         

In [2]:
import sqlite3
import pandas as pd

# DB 경로
DB_PATH = "db/seoul_bus_172.db"

# 연결
conn = sqlite3.connect(DB_PATH)

# 1) 테이블 목록 확인
tables = pd.read_sql("SELECT name FROM sqlite_master WHERE type='table';", conn)
print("📌 테이블 목록:")
print(tables)

# 2) daily_stop_hour에서 행 수 확인
row_count = pd.read_sql("SELECT COUNT(*) as cnt FROM daily_stop_hour;", conn)
print("\n📌 daily_stop_hour 총 레코드 수:")
print(row_count)

# 3) daily_stop_hour에서 앞부분 샘플 조회
sample = pd.read_sql("SELECT * FROM daily_stop_hour LIMIT 10;", conn)
print("\n📌 daily_stop_hour 샘플:")
print(sample)

conn.close()

📌 테이블 목록:
                              name
0                    dim_stop_link
1           fact_demand_daily_stop
2  fact_demand_monthly_hourly_stop
3                    dim_route_map
4             fact_ops_hourly_stop
5                  daily_stop_hour

📌 daily_stop_hour 총 레코드 수:
      cnt
0  586440

📌 daily_stop_hour 샘플:
        기준_날짜      노선ID     정류장_ID  시간  승차인원  하차인원 노선번호    역명  정류장_순서  운행횟수  \
0  2025-06-01  11110028  121500100   0   0.0   0.0  172  None       0     0   
1  2025-06-01  11110028  207000613   0   0.0   0.0  172  None       0     0   
2  2025-06-01  11110028  207000614   0   0.0   0.0  172  None       0     0   
3  2025-06-01  11110028  214000189   0   0.0   0.0  172  None       0     0   
4  2025-06-01  11110028  214000197   0   0.0   0.0  172  None       0     0   
5  2025-06-01  11110028  214000252   0   0.0   0.0  172  None       0     0   
6  2025-06-01  11110028  214000259   0   0.0   0.0  172  None       0     0   
7  2025-06-01  11110028  214001321   0   0

In [4]:
import sqlite3, pandas as pd
conn = sqlite3.connect("db/seoul_bus_172.db")
pd.read_sql("PRAGMA table_info(daily_stop_hour);", conn)

,cid,name,type,notnull,dflt_value,pk
0,0,기준_날짜,TEXT,1,None,0
1,1,노선ID,TEXT,0,None,0
2,2,정류장_ID,TEXT,1,None,0
3,3,시간,INTEGER,1,None,0
4,4,승차인원,REAL,0,None,0
5,5,하차인원,REAL,0,None,0
6,6,노선번호,TEXT,0,None,0
7,7,역명,TEXT,0,None,0
8,8,정류장_순서,INTEGER,0,None,0
9,9,운행횟수,INTEGER,0,None,0


In [5]:
col = "링크_구간거리_m"  # 확인된 이름으로 수정
q = f"""
SELECT 
  COUNT(*) AS total,
  SUM(CASE WHEN {col} IS NOT NULL THEN 1 ELSE 0 END) AS not_null,
  SUM(CASE WHEN {col} > 0 THEN 1 ELSE 0 END) AS gt0
FROM daily_stop_hour;
"""
pd.read_sql(q, conn)

,total,not_null,gt0
0,586440,0,0


In [6]:
col = "링크_구간거리_m"  # 확인된 이름
q = f"""
SELECT 기준_날짜, 노선ID, 정류장_ID, 시간, 정류장_순서, {col} AS 링크_m
FROM daily_stop_hour
WHERE {col} IS NOT NULL
  AND {col} > 0
  AND 정류장_순서 > 1
ORDER BY 기준_날짜, 시간, 정류장_순서
LIMIT 20;
"""
pd.read_sql(q, conn)

,기준_날짜,노선ID,정류장_ID,시간,정류장_순서,링크_m


In [7]:
import sqlite3, pandas as pd, numpy as np

DB = "db/seoul_bus_172.db"
MASTER = "/Users/minwoo/Desktop/softeer/DE5_Final_project/temp/서울시 노선 정류장마스터 정보_utf8.csv"

# 1) 마스터 로딩 (필수 컬럼: 정류장_ID, 정류장_순서, 링크_구간거리(m))
m = pd.read_csv(MASTER, dtype=str)
need = ["정류장_ID","정류장_순서","링크_구간거리(m)"]
missing = [c for c in need if c not in m.columns]
if missing:
    raise ValueError(f"마스터 CSV에 필요한 컬럼 없음: {missing}")

m["정류장_ID"]   = m["정류장_ID"].astype(str)
m["정류장_순서"] = pd.to_numeric(m["정류장_순서"], errors="coerce")
m["링크_구간거리_m"] = pd.to_numeric(m["링크_구간거리(m)"], errors="coerce")

# 같은 정류장 순서가 여러 번 있는 경우 평균으로 축약 (안전)
m_stop = (m.groupby(["정류장_ID","정류장_순서"], as_index=False)["링크_구간거리_m"]
            .mean())

# 2) DB 연결
conn = sqlite3.connect(DB)
cur = conn.cursor()

# 3) stop_id + stop_seq 매칭으로 업데이트
cur.execute("BEGIN")
for _, r in m_stop.iterrows():
    if pd.isna(r["정류장_순서"]) or pd.isna(r["링크_구간거리_m"]):
        continue
    cur.execute("""
        UPDATE daily_stop_hour
           SET 링크_구간거리_m = ?
         WHERE 정류장_ID = ?
           AND 정류장_순서 = ?
           AND (링크_구간거리_m IS NULL OR 링크_구간거리_m = 0)
    """, (float(r["링크_구간거리_m"]), str(r["정류장_ID"]), int(r["정류장_순서"])))
conn.commit()

# 4) 결과 확인 (샘플)
import pandas as pd
df_check = pd.read_sql("""
SELECT 기준_날짜, 노선ID, 정류장_ID, 정류장_순서, 시간, 링크_구간거리_m
FROM daily_stop_hour
WHERE 링크_구간거리_m IS NOT NULL AND 링크_구간거리_m > 0
ORDER BY 기준_날짜, 시간, 정류장_순서
LIMIT 20;
""", conn)
conn.close()
df_check

,기준_날짜,노선ID,정류장_ID,정류장_순서,시간,링크_구간거리_m
0,2025-06-01,11110028,106000469,1,0,57.285714
1,2025-06-01,11110028,110000308,1,0,11.400000
2,2025-06-01,11110028,117000132,1,0,27.857143
3,2025-06-01,11110028,123000440,1,0,85.000000
4,2025-06-01,11110028,100900029,2,0,181.000000
5,2025-06-01,11110028,100900032,2,0,69.000000
6,2025-06-01,11110028,100900034,2,0,486.000000
7,2025-06-01,11110028,100900035,2,0,179.000000
8,2025-06-01,11110028,100900036,2,0,199.000000
9,2025-06-01,11110028,100900038,2,0,200.000000
